# data processing

## json create

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
# package import

from data_pack import DataSetDefinition
import utils_cpu
import utils_gpu

from typing import List, Dict
from tqdm import tqdm

/home/work/anaconda3/envs/dilightnet/lib/python3.10/site-packages/_distutils_hack/__init__.py:54: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
/home/work/anaconda3/envs/dilightnet/lib/python3.10/site-packages/dust3r/cloud_opt/base_opt.py:275: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)


In [4]:
dataset_def = DataSetDefinition(
    processed_dir_suffix="128_v1"
)


In [34]:
dataset_def.processed_data_root_dir

'./data/processed'

In [5]:
step2_split = dataset_def.step2_dict_generator()

In [6]:
train_split = step2_split["train"]

In [6]:
from easydict import EasyDict
easy_dict = EasyDict(train_split[0])

NameError: name 'train_split' is not defined

In [9]:
easy_dict

{'image_path': './data/processed/train/images/obj_01_car/obj_01_car_004_CC7.png',
 'mask_path': './data/processed/train/masks/obj_01_car/obj_01_car_CC7.png',
 'viewpoint_id': 'CC7',
 'lighting_condition_id': '004',
 'image_id': 'obj_01_car',
 'output_dir': './data/processed/128_v1/train/hints',
 'fov': None,
 'mask_threshold': 0.25,
 'env_map': './data/generate_light_gt_sg/hdrs/004.hdr',
 'pls': [[0, 0, 0]],
 'use_gpu_for_rendering': True,
 'resolution': 128}

In [ ]:
radiance_result = utils_gpu.elem_generate_hint(train_split[0])

Rendering radiance hints for ./data/processed/train/images/obj_01_car/obj_01_car_004_CC7.png with viewpoint CC7 and lighting condition 004
Radiance hints not rendered to ./data/processed/128_v1/train/hints/obj_01_car/CC7/004/hint00_diffuse.png
Radiance hints not rendered to ./data/processed/128_v1/train/hints/obj_01_car/CC7/004/hint00_ggx0.05.png
Radiance hints already rendered to ./data/processed/128_v1/train/hints/obj_01_car/CC7/004/hint00_ggx0.13.png
Radiance hints already rendered to ./data/processed/128_v1/train/hints/obj_01_car/CC7/004/hint00_ggx0.34.png
>> Inference with model on 2 image pairs


  0%|          | 0/2 [00:00<?, ?it/s]

/home/work/anaconda3/envs/dilightnet/lib/python3.10/site-packages/dust3r/inference.py:44: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=bool(use_amp)):
/home/work/anaconda3/envs/dilightnet/lib/python3.10/site-packages/dust3r/model.py:199: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
/home/work/anaconda3/envs/dilightnet/lib/python3.10/site-packages/dust3r/inference.py:48: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
100%|██████████| 2/2 [00:02<00:00,  1.25s/it]


 init edge (0*,1*) score=np.float64(19.82822036743164)
 init loss = 0.0016863520722836256
Global alignement - optimizing for:
['pw_poses', 'im_depthmaps', 'im_poses', 'im_focals']


100%|██████████| 300/300 [00:06<00:00, 44.33it/s, lr=1.27413e-06 loss=0.00125575]


Mesh reconstructed with fov: 33.92671203613281
Data are loaded, start creating Blender stuff
glTF import finished in 0.25s
> /data2/code/diffusion-project/DiLightNet/demo/render_hints.py(54)configure_blender()
     53             import ipdb ; ipdb.set_trace()
---> 54             for d in bpy.context.preferences.addons["cycles"].preferences.devices:
     55                 if d.type == "CUDA":

<bpy_collection[3], CyclesPreferences.devices>
<bpy_struct, CyclesDeviceSettings("NVIDIA H100 80GB HBM3") at 0x3147c348>
<bpy_struct, CyclesDeviceSettings("Intel Xeon Platinum 8468") at 0x3147c3d0>
<bpy_struct, CyclesDeviceSettings("NVIDIA H100 80GB HBM3") at 0x3147c458>
*** IndexError: bpy_prop_collection[index]: index 3 out of range, size 3
<bpy_struct, CyclesDeviceSettings("NVIDIA H100 80GB HBM3") at 0x3147c348>
<bpy_struct, CyclesDeviceSettings("Intel Xeon Platinum 8468") at 0x3147c3d0>
<bpy_struct, CyclesDeviceSettings("NVIDIA H100 80GB HBM3") at 0x3147c348>
0
<bpy_struct, CyclesDeviceSetti

In [11]:
# directory structure -> list of data

step1_datadict_train_list: List[Dict[str, str]] = []
step1_datadict_eval_list: List[Dict[str, str]] = []

for each_class_name in dataset_def.class_name_list:
    for each_view_point in dataset_def.view_point_list:
        for each_light_condition in dataset_def.light_condition_list:

            image_path = DataSetDefinition.get_image_path(
                data_root_dir=dataset_def.raw_data_root_dir,
                class_name=each_class_name,
                light_condition=each_light_condition,
                view_point=each_view_point,
            )
            mask_path = DataSetDefinition.get_mask_path(
                data_root_dir=dataset_def.raw_data_root_dir,
                class_name=each_class_name,
                view_point=each_view_point,
            )

            parsed_name = DataSetDefinition.get_image_path(
                data_root_dir=dataset_def.raw_data_root_dir,
                class_name=each_class_name,
                light_condition=each_light_condition,
                view_point=each_view_point,
                parsed=True,
            )

            if parsed_name in dataset_def.eval_split:
                step1_datadict_eval_list.append(
                    {
                        "key": f"{each_class_name}_{each_light_condition}_{each_view_point}",
                        "image_path": image_path,
                        "mask_path": mask_path,
                    }
                )
            else:
                step1_datadict_train_list.append(
                    {
                        "key": f"{each_class_name}_{each_light_condition}_          {each_view_point}",
                        "image_path": image_path,
                        "mask_path": mask_path,
                    }
                )